In [1]:
import os
print("Current directory:", os.getcwd())
print("Files in folder:", os.listdir())
!pip install xlrd

Current directory: c:\Users\ruchi\Documents\Python_ironhack\week_2\data-cleaning-pandas
Files in folder: ['.git', 'cleaning_functions.py', 'data-cleaning-pandas.ipynb', 'Readme.md', 'Safe_Surf.pdf', 'shark_attack.xls']


In [ ]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your file
df = pd.read_excel('shark_attack.xls')

# Show the first few rows
print(df.head())


In [3]:
# Display shape and first few rows
print("Initial shape:", df.shape)
df.head(5)


Initial shape: (7027, 23)


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,6th July,2025.0,Provoked,USA,Massachusetts,Hoicks Hollow Nantucket Beach,Fishing,Unknown Male,M,21,...,Unknown,Johannes Marchand: Nantucket Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6th July,2025.0,Unprovoked,USA,Florida,New Smyrna Beach,Surfing,Unknown Male,M,40,...,Undetermined small shall shark,Todd Smith:Kevin McMurray Trackingsharks.com: ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4th July,2025.0,Unprovoked,South Africa,KwaZulu-Natal,Mfazazana Hibberdene,Fishing,Unknown male,M,37,...,Undetermined,Kevin McMurray Tracking sharks.com: East Coast...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29th June,2025.0,Unprovoked,Australia,NSW,Norries Headland Cabarita Beach,Swimming,Unknown teenage Male,M,16,...,2m Bull shark?,Simon De Marchi: Kevin McMurray Trackingsharks...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25th June,2025.0,Unprovoked,USA,Long Island NY,Wantagh,Swimming,Unknown Female,F,20,...,Sand Tiger shark,Scott Curatolo-Wagemann: News 12 Long Island: ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import pandas as pd
import numpy as np

# -------------------------
# Step 1: Clean dataset
# -------------------------

# Drop irrelevant columns safely
columns_to_drop = ['Case Number.1', 'href', 'href formula', 'original order', 'Unnamed: 22', 'Unnamed: 23', 'pdf', 'Case Number', 'Unnamed: 21']
for col in columns_to_drop:
    if col in df.columns:
        df.drop(columns=col, inplace=True)

df.columns = df.columns.str.strip()

# Dynamically find relevant columns
fatal_col = next((c for c in df.columns if 'fatal' in c.lower()), None)
country_col = next((c for c in df.columns if 'country' in c.lower()), None)
activity_col = next((c for c in df.columns if 'activity' in c.lower()), None)
date_col = next((c for c in df.columns if 'date' in c.lower()), None)

if not all([fatal_col, country_col, activity_col, date_col]):
    raise Exception("Dataset missing Fatal, Country, Activity, or Date column.")

df.drop_duplicates(inplace=True)

df[fatal_col] = df[fatal_col].astype(str).str.upper().str.strip()
df[country_col] = df[country_col].astype(str).str.title().str.strip()
df[activity_col] = df[activity_col].astype(str).str.title().str.strip()

# Parse dates with specified format (adjust '%Y-%m-%d' if needed)
df[date_col] = pd.to_datetime(df[date_col], format='%Y-%m-%d', errors='coerce').dt.date
df = df.dropna(subset=[date_col]).copy()

# Extract Year (keep for plotting)
df['Year'] = pd.to_datetime(df[date_col]).dt.year

# Managing null values for Age
df['Age'] = df['Age'].replace({
    'Teen': '15',
    'teen': '15',
    '30s': '35',
    '20s': '25',
    '40s': '45',
    '50s': '55',
    '60s': '65'
})
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
mean_age = df['Age'].mean()
df['Age'] = df['Age'].fillna(mean_age)

# Handle Year column zeros and NaNs
df['Year'] = df['Year'].replace(0.0, np.nan)
median_year = df['Year'].median()
df['Year'] = df['Year'].fillna(median_year)

# Standardize country names
df[country_col] = df[country_col].fillna('Unknown')
df[country_col] = df[country_col].apply(lambda x: x.strip().upper())

replacements = {
    'SUDAN?': 'SUDAN',
    'INDIAN OCEAN?': 'INDIAN OCEAN',
    'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES',
    'ST. MARTIN': 'ST MARTIN',
    'ST. MAARTIN': 'ST MARTIN'
}
df[country_col] = df[country_col].replace(replacements)

# Reset index starting from 1 for display
df = df.reset_index(drop=True)
df.index = df.index + 1

# -------------------------
# Step 2: Styled DataFrame
# -------------------------

top_risk_countries = ['USA', 'AUSTRALIA', 'SOUTH AFRICA']

def style_dataframe(df):
    def fatal_color(val):
        val = str(val).strip().upper()
        if val == 'Y':
            return 'background-color: red; color: white; font-weight: bold'
        elif val == 'N':
            return 'background-color: green; color: white; font-weight: bold'
        return ''

    def country_color(val):
        if val in top_risk_countries:
            return 'background-color: orange; color: black; font-weight: bold'
        return ''

    def activity_color(val):
        if isinstance(val, str) and val.lower() == 'surfing':
            return 'background-color: lightblue; font-weight: bold'
        return ''

    styled = (df.style
        .set_table_attributes('style="width:100%;border-collapse:collapse;border:1px solid black"')
        .set_table_styles([
            {'selector': 'th',
             'props': [('background-color', '#007BFF'),
                       ('color', 'white'),
                       ('text-align', 'center'),
                       ('border', '1px solid black'),
                       ('padding', '8px')]},
            {'selector': 'td',
             'props': [('border', '1px solid black'),
                       ('text-align', 'center'),
                       ('padding', '6px')]},
        ])
        .set_properties(**{'text-align': 'center', 'border': '1px solid black'})
    )

    if fatal_col in df.columns:
        styled = styled.map(fatal_color, subset=[fatal_col])
    if country_col in df.columns:
        styled = styled.map(country_color, subset=[country_col])
    if activity_col in df.columns:
        styled = styled.map(activity_color, subset=[activity_col])

    return styled

# Display styled preview (first 20 rows)
styled_df = style_dataframe(df.head(5))
display(styled_df)

# -------------------------
# Step 3: Plotly Visualizations
# -------------------------
# Add your plotting code here, now you can use df['Year'] for visualizations


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source
1,2025-06-11,2025,Unprovoked,USA,Florida,Boca Grande,Snorkeling,Leah Lendel,F,9.000000,Right hand almost torn off,N,1211 hrs,Bull shark,Kevin McMurray Trackingsharks.com: James Kingsley: Fox 4 News
2,2025-05-29,2025,Unprovoked,USA,North Carolina,Sunset Beach,Swimming,Sean Barton,M,26.000000,Gash to lower left leg,N,1100hrs,Not stated,Kevin McMurray Trackingsharks.com: Clay Crewell:Scott Curatolo-Wageman
3,2025-05-26,2025,Unprovoked,VANUATU,South Santo,Espiitu Santo Island,Swimming,Tumas,M,14.000000,Multiple injuries to body one hand and leg bitten off,Y,pm,Not stated,Kevin McMurray Trackingsharks.com
4,2025-05-15,2025,Unprovoked,AUSTRALIA,South Australia,Port Noarlunga,Swimming,Richard Vinall,M,66.000000,Injury to thigh,N,0945hrs,Not stated,Simon DeMarchi: Todd Smith: 9 News:ABC News
5,2025-05-08,2025,Unprovoked,JAMAICA,Montego Bay,Rose Hall beach,Paddling,Rachel Smith,F,26.000000,minor injuries to fingers of left hand,N,Not stated,Small shark 1m (3ft) long,The sun News: Metro: Kevin McMurray Tracking sharks.com


In [6]:
import plotly.express as px

# Assume df is your cleaned DataFrame and fatal_col, country_col, activity_col, date_col are detected

print("Columns in dataset:", df.columns.tolist())

# 1. Fatal vs Non-Fatal Pie Chart
fig1 = px.pie(df, names=fatal_col, title='Fatal vs Non-Fatal Shark Attacks')
fig1.show()

# 2. Top 10 Countries by Shark Attacks
top_countries = df[country_col].value_counts().nlargest(5)
fig2 = px.bar(x=top_countries.index, y=top_countries.values,
              title='Top 5 Countries by Shark Attacks',
              labels={'x': 'Country', 'y': 'Number of Attacks'})
fig2.show()

# 3. Shark Attacks Over the Years
df_filtered = df[df['Year'] >= 1700]
attacks_per_year = df_filtered['Year'].value_counts().sort_index()
fig3 = px.line(x=attacks_per_year.index, y=attacks_per_year.values,
               title='Shark Attacks Over the Years',
               labels={'x': 'Year', 'y': 'Number of Attacks'})
fig3.show()

# 4. Top 10 Activities During Shark Attacks
top_activities = df[activity_col].value_counts().nlargest(8)
fig4 = px.bar(x=top_activities.index, y=top_activities.values,
              title='Top 8 Activities During Shark Attacks',
              labels={'x': 'Activity', 'y': 'Number of Attacks'})
fig4.show()

# 5. Sunburst Chart: Country → Activity → Fatal
df_sunburst = df.dropna(subset=[country_col, activity_col, fatal_col])
fig5 = px.sunburst(df_sunburst, path=[country_col, activity_col, fatal_col],
                   title='Sunburst: Country → Activity → Fatal')
fig5.show()


Columns in dataset: ['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species', 'Source']
